Imports

In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
import time
import os
import copy
from PIL import Image
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time

# Set GPU Settings
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['TORCH_USE_CUDA_DSA'] = '1'

#TURN OFF WHEN ACTUALLY TESTING CODE
import warnings
warnings.filterwarnings("ignore")

Data Setup

In [2]:
# Define CarDataSet Class
class CarDataSet():

    # Define The Initialization
    def __init__(self, csv_file, root_dir, transform=None, target_transform=None):
        self.cars = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.target_transform = target_transform
        self.resize = transforms.Resize((150,150))  # Resize images to a uniform size

    # Define The Length Function
    def __len__(self):
        return len(self.cars)

    # Define The Get Item Function
    def __getitem__(self, idx):

        # Pull The Image And Check Settings
        img_name = os.path.join(self.root_dir, self.cars.iloc[idx, 0])

        image = Image.open(img_name)

        if image.mode != 'RGB':
          image = image.convert('RGB')

        # Pull The Label, -1 To Normalize To 0
        label = (self.cars.iloc[idx, 5]) - 1

        if self.transform:
            image = self.transform(image)

        # Define The Dictionary
        sample = {'image': image, 'cars': label}

        # Return
        return sample

In [3]:
# Define Transform
transform = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()])
    
# Load The Data
train_data = CarDataSet(csv_file='../SynchronizationProject/stanford_cars_eec174/train_make.csv',
                                        root_dir='../SynchronizationProject/stanford_cars_eec174/images/train', transform=transform)
test_data = CarDataSet(csv_file='../SynchronizationProject/stanford_cars_eec174/val_make.csv',
                                        root_dir='../SynchronizationProject/stanford_cars_eec174/images/val', transform=transform)

In [4]:
def show_imgs(dataloader):
    # Load Names From The File
    with open('../SynchronizationProject/stanford_cars_eec174/names_make.txt', 'r') as file:
        names = file.read().splitlines()

    # Retrieve The Images And Labels
    dataiter = iter(dataloader)
    samples = next(dataiter)

    images_show_img, labels_show_img = samples['image'], samples['cars']

    # Grab Ten Random Indices
    shuffled_indices = np.random.permutation(len(images_show_img))
    indices = shuffled_indices[:10]

    # Create Subplots
    figure, axes = plt.subplots(1, 10, figsize=(20, 10))

    for i, ax in zip(indices, axes):

        # Rearrange Dimensions For Display
        cur_image = images_show_img[i].permute(1, 2, 0)

        # Convert The Label To An Integer
        label_index = int(labels_show_img[i].item())

        # Assign The Name Corresponding To The Label Index
        cur_label = names[label_index]

        # Display The Image
        ax.imshow(cur_image)

        # Display The Label As Title
        ax.set_title(cur_label)

        # Turn Off The Axis
        ax.axis('off')

    plt.show()

# Test Accuracy
def test_accuracy(model, test_loader_internal, passed_device, loss_fn):

    # Set Parameters
    model.to(passed_device)
    correct = 0
    total = 0
    run = 0
    val_loss = 0

    # Run Tests
    with torch.no_grad():
        for test_data_internal in test_loader_internal:
            images_test_acc, labels_test_acc = test_data_internal['image'].cuda(), test_data_internal['cars'].cuda()
            outputs_test_acc = model(images_test_acc)
            _, predicted_test_acc = torch.max(outputs_test_acc.data, 1)
            val_loss += (loss_fn(outputs_test_acc, labels_test_acc)).item()
            total += labels_test_acc.size(0)
            correct += (predicted_test_acc == labels_test_acc).sum().item()
            run += 1

    # Return
    return (100 * correct / total), val_loss/run


# Plotting Function
def general_plot(data_dict1: dict, data_dict2: dict, param: str, label1: str, label2: str, ylabel: str, reduce_noise: bool):
    param_list1 = data_dict1[param]
    param_list2 = data_dict2[param]

    if reduce_noise:
        param_list1.pop(0)
        param_list2.pop(0)

    # Prepare Plot
    xs = [x for x in range(min(len(param_list1), len(param_list2)))]
    plt.figure(figsize=(20, 10))
    plt.plot(xs, param_list1[:len(xs)], label=label1)
    plt.plot(xs, param_list2[:len(xs)], label=label2)
    plt.xlabel('Iterations (in batches)')
    plt.ylabel(ylabel)
    plt.title(f'Training and Validation {ylabel} Curve')
    plt.legend()
    plt.show()
    
    
# Plot Learning Curves
def plot_learning_curve(train_history_plt_curve: dict, val_history_plt_curve: dict, reduce_noise=True):

    # Plot The Info
    general_plot(train_history_plt_curve, val_history_plt_curve, 'loss', 'Training Loss', 'Validation Loss', 'Loss', reduce_noise)
    general_plot(train_history_plt_curve, val_history_plt_curve, 'accuracy', 'Training Accuracy', 'Validation Accuracy', 'Accuracy', reduce_noise)
    
# Define and use confusion matrix
def plot_confusion_matrix(y_true, y_pred, class_names_confusion):

    # Prepare Plot And Matrix
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(20, 20))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion Matrix Frozen')
    plt.colorbar()
    tick_marks = np.arange(len(class_names_confusion))
    plt.xticks(tick_marks, class_names_confusion, rotation=45)
    plt.yticks(tick_marks, class_names_confusion)

    fmt = 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], fmt),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")

    # Show Everything
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.tight_layout()
    plt.show()


# Define Train
def train(model, loss_fn, optimizer_train, train_loader_train, val_loader, num_epochs_train, device_train):

    # Set Parameters
    model.train()
    BLUE = '\033[94m'
    RESET = '\033[0m'

    train_history_train = {'loss': [], 'accuracy': []}
    val_history_train = {'loss': [], 'accuracy': []}

    total_start_time = time.time()

    # Iterate Through All Epochs
    for epoch in range(num_epochs_train):

        # For Loop Parameters
        start_time = time.time()
        total = 0
        correct = 0
        train_loss = 0
        train_correct = 0
        train_total = 0

        # Iterate Through The Training Dataset
        for i, data_train in enumerate(train_loader_train, 0):
            
            # Flatten Images And Load Data
            images_train, labels_train = data_train['image'].cuda(), data_train['cars'].cuda()

            # Zero Collected Gradients At Each Step (basically cleaning)
            optimizer_train.zero_grad()

            # Forward Propagate
            outputs_train = model(images_train)

            # Calculate Loss
            loss = loss_function(outputs_train, labels_train)

            # Back Propagate
            loss.backward()

            # Update Weigh Gradsts
            optimizer_train.step()

            # Calculate Accuracy
            _, predicted_train = torch.max(outputs_train.data, 1)
            total += labels_train.size(0)
            correct += (predicted_train == labels_train).sum().item()

            train_loss += loss.item()
            train_correct += 100 * correct / total
            train_total += 1
            
        # Evaluate Model
        model.eval()
        with torch.no_grad():

            val_acc, val_loss = test_accuracy(model, val_loader, device_train, loss_fn)

            val_history_train['loss'].append(val_loss)
            val_history_train['accuracy'].append(val_acc)

            train_history_train['loss'].append(train_loss / train_total)
            train_history_train['accuracy'].append(train_correct / train_total)

            train_loss = 0
            train_correct = 0
            train_total = 0
        model.train()


        # At End Of Each Epoch Print Duration And Accuracy
        print(f'{BLUE}Epoch [{epoch+1}/{num_epochs_train}]:'f' Duration: {round(time.time() - start_time, 2)}s |'f' Train Acc: {round(train_history_train["accuracy"][-1], 2)} |'f' Train Loss: {round(train_history_train["loss"][-1], 5)}'f' Val Acc: {round(val_history_train["accuracy"][-1], 2)} |'f' Val Loss: {round(val_history_train["loss"][-1], 5)}{RESET}')
        print('------------------------------------------------------------------------------------------------------------')

        # Save Checkpoint
        #PATH = '/content/drive/My Drive/WEIGHTSAVES/run' + str('78.8save') + 'epoch' + str(epoch+1) + 'ta' + str(train_history_train["accuracy"][-1])[0:6] + 'tl' + str(train_history_train["loss"][-1])[0:6] + 'va' + str(val_history_train["accuracy"][-1])[0:6] + 'vl' + str(val_history_train["loss"][-1])[0:6] + '.pth'
        #print(PATH)
        #torch.save(net.state_dict(), PATH)

    # Print Final Statistics
    #print(f'{BLUE}Total Duration:{round(time.time() - total_start_time, 2)}s |',f'Final Train Acc: {round(train_history_train["accuracy"][-1], 2)} |',f'Final Train Loss: {round(train_history_train["loss"][-1], 5)} |',f'Final Val Acc: {round(val_history_train["accuracy"][-1], 2)} |',f'Final Val Loss: {round(val_history_train["loss"][-1], 5)}{RESET}')

    # Return
    return train_history_train, val_history_train

In [5]:
def to_numpy(tensor):
    return tensor.cpu().numpy()
 
def maximum_weight_aggregation(models_states):
    aggregated_weights = {}
    for param_name in models_states[0]:
        all_weights = [to_numpy(model_state[param_name]) for model_state in models_states]
        aggregated_weights[param_name] = torch.tensor(np.max(all_weights, axis=0))
    return aggregated_weights, "MaximumWeight"

def minimum_weight_aggregation(models_states):
    aggregated_weights = {}
    for param_name in models_states[0]:
        all_weights = [to_numpy(model_state[param_name]) for model_state in models_states]
        aggregated_weights[param_name] = torch.tensor(np.min(all_weights, axis=0))
    return aggregated_weights, "MinimumWeight"

def mean_aggregation(models_states):
    aggregated_weights = {}
    for param_name in models_states[0]:
        all_weights = [to_numpy(model_state[param_name]) for model_state in models_states]
        aggregated_weights[param_name] = torch.tensor(np.mean(all_weights, axis=0))
    return aggregated_weights, "MeanWeight"

def median_aggregation(models_states):
    aggregated_weights = {}
    for param_name in models_states[0]:
        all_weights = [to_numpy(model_state[param_name]) for model_state in models_states]
        aggregated_weights[param_name] = torch.tensor(np.median(all_weights, axis=0))
    return aggregated_weights, "MedianWeight"

Unchanging Definitions

#No Pretrained
    "0": [1, 1, 1, False, 1], #Run 0: batch size 32, learning rate 0.0001, 1 GPU, 50 epochs
    "1": [1, 1, 3, False, 1], #Run 1: batch size 32, learning rate 0.0001, 3 GPUs, 50 Epochs
    "2": [1, 1, 6, False, 1], #Run 2: batch size 32, learning rate 0.0001, 6 GPU, 50 Epochs
    "3": [1, 1, 10, False, 1], #Run 3: batch size 32, learning rate 0.0001, 10 GPUs, 50 Epochs
    
    "4": [1, 10, 1, False, 1], #Run 4: batch size 32, learning rate 0.001, 1 GPU, 50 epochs
    "5": [1, 10, 3, False, 1], #Run 5: batch size 32, learning rate 0.001, 3 GPUs, 50 Epochs
    "6": [1, 10, 6, False, 1], #Run 6: batch size 32, learning rate 0.001, 6 GPU, 50 Epochs
    "7": [1, 10, 10, False, 1], #Run 7: batch size 32, learning rate 0.001, 10 GPUs, 50 Epochs
    
    
    
    
    
    "8": [2, 1, 1, False, 1], #Run 8: batch size 64, learning rate 0.0001, 1 GPU, 50 epochs
    "9": [2, 1, 3, False, 1], #Run 9: batch size 64, learning rate 0.0001, 3 GPUs, 50 Epochs
    "10": [2, 1, 6, False, 1], #Run 10: batch size 64, learning rate 0.0001, 6 GPU, 50 Epochs
    "11": [2, 1, 10, False, 1], #Run 11: batch size 64, learning rate 0.0001, 10 GPUs, 50 Epochs
    
    "12": [2, 10, 1, False, 1], #Run 12: batch size 64, learning rate 0.001, 1 GPU, 50 epochs
    "13": [2, 10, 3, False, 1], #Run 13: batch size 64, learning rate 0.001, 3 GPUs, 50 Epochs
    "14": [2, 10, 6, False, 1], #Run 14: batch size 64, learning rate 0.001, 6 GPU, 50 Epochs
    "15": [2, 10, 10, False, 1], #Run 15: batch size 64, learning rate 0.001, 10 GPUs, 50 Epochs






"16": [1, 1, 1, False, 2], #Run 16: batch size 32, learning rate 0.0001, 1 GPU, 100 epochs
"17": [1, 1, 3, False, 2], #Run 17: batch size 32, learning rate 0.0001, 3 GPUs, 100 Epochs
"18": [1, 1, 6, False, 2], #Run 18: batch size 32, learning rate 0.0001, 6 GPU, 100 Epochs
"19": [1, 1, 10, False, 2], #Run 19: batch size 32, learning rate 0.0001, 10 GPUs, 100 Epochs
"20": [1, 10, 1, False, 2], #Run 20: batch size 32, learning rate 0.001, 1 GPU, 100 epochs
"21": [1, 10, 3, False, 2], #Run 21: batch size 32, learning rate 0.001, 3 GPUs, 100 Epochs
"22": [1, 10, 6, False, 2], #Run 22: batch size 32, learning rate 0.001, 6 GPU, 100 Epochs
"23": [1, 10, 10, False, 2], #Run 23: batch size 32, learning rate 0.001, 10 GPUs, 100 Epochs
"24": [2, 1, 1, False, 2], #Run 24: batch size 64, learning rate 0.0001, 1 GPU, 100 epochs
"25": [2, 1, 3, False, 2], #Run 25: batch size 64, learning rate 0.0001, 3 GPUs, 100 Epochs
"26": [2, 1, 6, False, 2], #Run 26: batch size 64, learning rate 0.0001, 6 GPU, 100 Epochs
"27": [2, 1, 10, False, 2], #Run 27: batch size 64, learning rate 0.0001, 10 GPUs, 100 Epochs
"28": [2, 10, 1, False, 2], #Run 28: batch size 64, learning rate 0.001, 1 GPU, 100 epochs
"29": [2, 10, 3, False, 2], #Run 29: batch size 64, learning rate 0.001, 3 GPUs, 100 Epochs
"30": [2, 10, 6, False, 2], #Run 30: batch size 64, learning rate 0.001, 6 GPU, 100 Epochs
"31": [2, 10, 10, False, 2] #Run 31: batch size 64, learning rate 0.001, 10 GPUs, 100 Epochs

In [6]:
#[GPUcount num, pretrained (True=yes, False=no)] 

operations_dictionary = {
    "0":[1, True, 'PreSave15', 0],
    "1":[3, True, 'PreSave15', 3]
                         }
# Dataset Size
DATASET_SIZE = 8192

# Define Parameters
input_size = (768 * 1024)
num_classes = 49

InfoSave = pd.DataFrame(columns = ["Run", "BatchSize", "LearningRate", "Epochs", "TrainAccuracy", "TrainLoss", "TrainHistory", "ValAccuracy", "ValLoss", "ValHistory", "GPUCount", "PreTrain", "MergeType", "TrainTime", "SavePathInput", "SavePathOutput"])

In [7]:
# net = models.resnet50()
# net.fc = nn.Linear(net.fc.in_features, 49)
# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# net.to(device)
# 
# batch_size = 32
# 
# # Import To Dataloaders
# train_loader = torch.utils.data.DataLoader(dataset = train_data, batch_size = batch_size, shuffle = True)
# test_loader = torch.utils.data.DataLoader(dataset = test_data, batch_size = batch_size, shuffle = True)
# 
# # Specify Factors
# lr = 0.0001
# 
# # Loss Function and Optimizer
# loss_function = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(net.parameters(), lr=lr)
# 
# # Run Train
# train_history, val_history = train(net, loss_function, optimizer, train_loader, test_loader, 10, device)
# 
# PATH = '../SynchronizationProject/SaveData/PreSave15.pth'
# torch.save(net.state_dict(), PATH)

Epoch [1/10]: Duration: 111.61s | Train Acc: 10.08 | Train Loss: 3.49497 Val Acc: 11.24 | Val Loss: 3.46975
------------------------------------------------------------------------------------------------------------
Epoch [2/10]: Duration: 111.53s | Train Acc: 12.17 | Train Loss: 3.40937 Val Acc: 8.71 | Val Loss: 3.54531
------------------------------------------------------------------------------------------------------------
Epoch [3/10]: Duration: 114.72s | Train Acc: 11.55 | Train Loss: 3.34455 Val Acc: 12.61 | Val Loss: 3.39154
------------------------------------------------------------------------------------------------------------
Epoch [4/10]: Duration: 113.13s | Train Acc: 14.19 | Train Loss: 3.27431 Val Acc: 11.99 | Val Loss: 3.35498
------------------------------------------------------------------------------------------------------------
Epoch [5/10]: Duration: 111.28s | Train Acc: 16.04 | Train Loss: 3.18982 Val Acc: 13.81 | Val Loss: 3.25399
-------------------------

Model Setup and Training

In [9]:
# Run Each Set of Models 
for run_number_key in operations_dictionary:
    print(run_number_key)
    
    # Specify Factors
    lr = 0.0001
    batch_size = 32
    num_epochs = 30
    
    # Import To Dataloaders
    train_loader = torch.utils.data.DataLoader(dataset = train_data, batch_size = batch_size, shuffle = True)
    test_loader = torch.utils.data.DataLoader(dataset = test_data, batch_size = batch_size, shuffle = True)
        
    # Single GPU
    if (operations_dictionary[run_number_key])[0] == 1:
        
        start_time = time.time()
        
        # Model Definition and Final Layer Edit
        net = models.resnet50()
        net.fc = nn.Linear(net.fc.in_features, 49)
        
        saved_model_path = ""
        
        if (operations_dictionary[run_number_key])[1]:
            # IF NEEDED TO IMPORT
            saved_model_path = '../SynchronizationProject/SaveData/' + str((operations_dictionary[run_number_key])[2]) + '.pth'
            net.load_state_dict(torch.load(saved_model_path))
        
        # Load Model Onto GPU
        device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        net.to(device)
        
        # Loss Function and Optimizer
        loss_function = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(net.parameters(), lr=lr)
        
        # Run Train
        train_history, val_history = train(net, loss_function, optimizer, train_loader, test_loader, num_epochs, device)
        
        # Save Model
        PATH = '../SynchronizationProject/SaveData/Weights/' + str(run_number_key) + '.pth'
        torch.save(net.state_dict(), PATH)
        
        total_time = time.time() - start_time
        
        trainacc, trainloss = test_accuracy(net, train_loader, device, loss_function)
        valacc, valloss = test_accuracy(net, test_loader, device, loss_function)
        
        new_row_data = {
                "Run": run_number_key,
                "BatchSize": batch_size,
                "LearningRate": lr,
                "Epochs": num_epochs,
                "TrainAccuracy": trainacc,
                "TrainLoss": trainloss,
                "ValAccuracy": valacc,
                "ValLoss": valloss,
                "GPUCount": 1,
                "PreTrain": (operations_dictionary[run_number_key])[1],
                "MergeType": "NONE",
                "TrainTime": total_time,
                "SavePathInput": saved_model_path,
                "SavePathOutput": PATH
        }
            
        # Adding the new row to the DataFrame
        InfoSave.loc[len(InfoSave)] = new_row_data
    
    # Multiple GPU
    else:
        #amount of gpus to run
        amount_of_GPUs_to_run = (operations_dictionary[run_number_key])[0]
        
        #make an array of input data to access
        inputDataTrain = []
        for datasetNum in range(amount_of_GPUs_to_run):
            start_idx = (len(train_loader.dataset) // amount_of_GPUs_to_run) * datasetNum
            
            if datasetNum < (amount_of_GPUs_to_run - 1):
                end_idx = (len(train_loader.dataset) // amount_of_GPUs_to_run) * (datasetNum + 1)
            else:
                end_idx = len(train_loader.dataset)
                
            split_data_set = torch.utils.data.Subset(train_loader.dataset, range(start_idx, end_idx))
            split_train_loader = torch.utils.data.DataLoader(dataset=split_data_set, batch_size=batch_size, shuffle=True)
            
            inputDataTrain.appenmd(split_train_loader)
        
        #iterate through each type of synchronizatyion, there are 4
            for type in range(4):
                
                start_time = time.time()
                
                # Model Definition and Final Layer Edit
                net = models.resnet50()
                net.fc = nn.Linear(net.fc.in_features, 49)
                
                saved_model_path = ""
                
                if (operations_dictionary[run_number_key])[1]:
                    # IF NEEDED TO IMPORT
                    saved_model_path = '../SynchronizationProject/SaveData/' + str((operations_dictionary[run_number_key])[2]) + '.pth'
                    net.load_state_dict(torch.load(saved_model_path))
                    
                # Loss Function and Optimizer
                loss_function = nn.CrossEntropyLoss()
                optimizer = torch.optim.Adam(net.parameters(), lr=lr)
                
                #iterate through all the syncs
                for syncNum in range((operations_dictionary[run_number_key])[3]):
                    weightsList = []
                    #iterate through all the GPU trains
                    for gpuNumRun in range(amount_of_GPUs_to_run):
                        # Load Model Onto GPU
                        device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
                        net.to(device)
                        
                        #train
                        train_history, val_history = train(net, loss_function, optimizer, inputDataTrain[gpuNumRun], test_loader, num_epochs/(operations_dictionary[run_number_key])[3], device)
                        
                
                    #sync them
                    if type == 0:
                        weightOutput, name = maximum_weight_aggregation(weightsList)
                    elif type == 1:
                        weightOutput, name = minimum_weight_aggregation(weightsList)
                    elif type == 2:
                        weightOutput, name = median_aggregation(weightsList)
                    elif type == 3:
                        weightOutput, name = mean_aggregation(weightsList)
                        
                    net.load_state_dict(weightOutput)
                
                #save final sync output and move onto next sync
                
                
                # Save Model
                PATH = '../SynchronizationProject/SaveData/Weights/' + str(run_number_key) + str(name) + '.pth'
                torch.save(net.state_dict(), PATH)
            
                total_time = time.time() - start_time
                
                trainacc, trainloss = test_accuracy(net, train_loader, device, loss_function)
                valacc, valloss = test_accuracy(net, test_loader, device, loss_function)
                
                new_row_data = {
                        "Run": run_number_key,
                        "BatchSize": batch_size,
                        "LearningRate": lr,
                        "Epochs": num_epochs,
                        "TrainAccuracy": trainacc,
                        "TrainLoss": trainloss,
                        "ValAccuracy": valacc,
                        "ValLoss": valloss,
                        "GPUCount": 1,
                        "PreTrain": (operations_dictionary[run_number_key])[1],
                        "MergeType": "NONE",
                        "TrainTime": total_time,
                        "SavePathInput": saved_model_path,
                        "SavePathOutput": PATH
                }
                    
                # Adding the new row to the DataFrame
                InfoSave.loc[len(InfoSave)] = new_row_data

file_path = '../SynchronizationProject/SaveData/DataFrames/InfoSave.csv'

InfoSave.to_csv(file_path, index=False)

Evaluate The Model

In [9]:
file_path = '../SynchronizationProject/SaveData/DataFrames/InfoSave.csv'

InfoSave = pd.read_csv(file_path)

In [12]:
InfoSave

,Run,BatchSize,LearningRate,Epochs,TrainAccuracy,TrainLoss,TrainHistory,ValAccuracy,ValLoss,ValHistory,GPUCount,GPUNumber,PreTrain,TrainTime,SavePathInput,SavePathOutput
0,0,32,0.0001,50,98.891674,0.046048,"{'loss': [3.4871723436841777, 3.42232744647007...",18.830361,5.840313,"{'loss': [3.490916973666141, 3.408613242601093...",1,1,False,6513.744140,NaN,../SynchronizationProject/SaveData/Weights/0.pth
1,1,32,0.0001,50,99.954355,0.006749,"{'loss': [3.5152197248795454, 3.43448365435880...",12.691829,5.167353,"{'loss': [3.5044365682099996, 3.46959972381591...",3,1,False,2789.132076,NaN,../SynchronizationProject/SaveData/Weights/1T0...
2,1,32,0.0001,50,99.989637,0.011231,"{'loss': [3.5432352767271156, 3.46236025024862...",11.489009,4.928459,"{'loss': [3.492495696795614, 3.476980432083732...",3,2,False,2805.588549,NaN,../SynchronizationProject/SaveData/Weights/1T1...
3,1,32,0.0001,50,99.990760,0.004735,"{'loss': [3.547464345483219, 3.464133195316090...",11.198673,5.037350,"{'loss': [3.549210316256473, 3.463777341340717...",3,3,False,2776.819473,NaN,../SynchronizationProject/SaveData/Weights/1T2...
4,2,32,0.0001,50,99.991356,0.025019,"{'loss': [3.5846809398296267, 3.42887252985045...",8.917462,5.132797,"{'loss': [3.548779399771439, 3.598160019046382...",6,1,False,1857.319781,NaN,../SynchronizationProject/SaveData/Weights/2T0...
5,2,32,0.0001,50,99.996572,0.023236,"{'loss': [3.5873300252958784, 3.44102504641510...",9.498134,5.192931,"{'loss': [3.5128325819969177, 3.52327824580042...",6,2,False,1863.976217,NaN,../SynchronizationProject/SaveData/Weights/2T1...
6,2,32,0.0001,50,99.766458,0.029173,"{'loss': [3.5875701793404513, 3.41547107142071...",7.839071,5.590741,"{'loss': [3.5278035370927108, 3.51810194002954...",6,3,False,1875.593079,NaN,../SynchronizationProject/SaveData/Weights/2T2...
7,2,32,0.0001,50,81.397344,0.835543,"{'loss': [3.6037130965742956, 3.48660362598507...",3.649938,9.772663,"{'loss': [3.5434660221401013, 3.52629286364505...",6,4,False,1859.868212,NaN,../SynchronizationProject/SaveData/Weights/2T3...
8,2,32,0.0001,50,100.000000,0.005080,"{'loss': [3.582980466443439, 3.435957720113355...",8.087930,4.891955,"{'loss': [3.5835642438185844, 3.55822451804813...",6,5,False,1857.146929,NaN,../SynchronizationProject/SaveData/Weights/2T4...
9,2,32,0.0001,50,99.510467,0.040453,"{'loss': [3.6127316729966985, 3.47479107213574...",8.087930,5.406475,"{'loss': [3.551482981757114, 3.532131151149147...",6,6,False,1851.784572,NaN,../SynchronizationProject/SaveData/Weights/2T5...


In [ ]:
# Plot Learning Curves
plot_learning_curve(train_history, val_history)

In [ ]:
# Put Model In Eval Mode
net.eval()

# Create Lists To Store The Predictions And True Labels
predictions = []
true_labels = []

# Iterate Through The Test Data
with torch.no_grad():
    for data in test_loader:

        # Load Data
        images, labels = data['image'].cuda(), data['cars'].cuda()
        outputs = net(images)

        true_labels.extend(labels.cpu().numpy())

        # Get Class With Highest Probability As Predicted Class
        _, predicted = torch.max(outputs, 1)

        # Convert Predictions To List And Append
        predictions.extend(predicted.cpu().numpy())

# Convert Predictions List And true_labels To NumPy Arrays
predictions = np.array(predictions)
true_labels = np.array(true_labels)

# Pull Class Names
class_names = (['AM'] + (pd.read_csv('../SynchronizationProject/stanford_cars_eec174/names_make.txt')['AM'].to_list()))

# Plot Everything
plot_confusion_matrix(true_labels, predictions, class_names)